# System-Theoretic Methods with pyMOR

## Linear Time-Invariant (LTI) Systems

$$
\newcommand{\imag}{\boldsymbol{\imath}}
\newcommand{\tran}{^{\operatorname{T}}}
\newcommand{\mtran}{^{-\!\operatorname{T}}}
\begin{align*}
  E \dot{x}(t) & = A x(t) + B u(t), \quad x(0) = 0, \\
  y(t) & = C x(t) + D u(t),
\end{align*}
$$

- $u(t) \in \mathbb{R}^m$ is the input,
- $x(t) \in \mathbb{R}^n$ is the state,
- $y(t) \in \mathbb{R}^p$ is the output,
- $E, A \in \mathbb{R}^{n \times n}$,
- $B \in \mathbb{R}^{n \times m}$,
- $C \in \mathbb{R}^{p \times n}$,
- $D \in \mathbb{R}^{p \times m}$.

We'll assume that:

- $D = 0$,
- $E$ is invertible,
- $E^{-1} A$ is Hurwitz (all the eigenvalues have negative real parts).

## Example (Mass-Spring-Damper Chain)

Equations:
$$
\begin{align*}
  m \ddot{q}_1(t)
  + d (2 \dot{q}_1(t) - \dot{q}_2(t))
  + k (2 q_1(t) - q_{2}(t))
  & = u(t), \\
  m \ddot{q}_i(t)
  + d (2 \dot{q}_i(t) - \dot{q}_{i - 1}(t) - \dot{q}_{i + 1}(t))
  + k (2 q_i(t) - q_{i - 1}(t) - q_{i + 1}(t))
  & = 0,
  & i = 2, \ldots, n - 1, \\
  m \ddot{q}_n(t)
  + d (2 \dot{q}_n(t) - \dot{q}_{n - 1}(t))
  + k (2 q_n(t) - q_{n - 1}(t))
  & = 0, \\
  y(t) & = q_n(t)
\end{align*}
$$

Matrix form:
$$
\begin{align*}
  M_{so} \ddot{q}(t)
  + D_{so} \dot{q}(t)
  + K_{so} q(t)
  & = B_{so} u(t), \\
  y(t)
  & = C_{so} q(t)
\end{align*}
$$
where
$$
M_{so} = m I, \quad
D_{so} = d L, \quad
K_{so} = k L, \quad
L =
\begin{bmatrix}
  2 & -1 \\
  -1 & 2 & -1 \\
  & -1 & \ddots & \ddots \\
  & & \ddots & \ddots & -1 \\
  & & & -1 & 2
\end{bmatrix}, \quad
B_{so} = e_1, \quad
C_{so} = e_n^T.
$$

In first-order form:
$$
x(t) =
\begin{bmatrix}
  q(t) \\
  \dot{q}(t)
\end{bmatrix}, \quad
E =
\begin{bmatrix}
  I & 0 \\
  0 & M_{so}
\end{bmatrix}, \quad
A =
\begin{bmatrix}
  0 & I \\
  -K_{so} & -D_{so}
\end{bmatrix}, \quad
B =
\begin{bmatrix}
  0 \\
  B_{so}
\end{bmatrix}, \quad
C =
\begin{bmatrix}
  C_{so} & 0
\end{bmatrix}, \quad
$$

## Building a Full-Order Model

### Matrices

In [ ]:
import numpy as np
import scipy.sparse as sps

In [ ]:
n = 1000
m = 1
d = 10
k = 1
Mso = m * sps.eye(n, format='csc')
L = sps.diags([(n-1)*[-1], n*[2], (n-1)*[-1]], [-1, 0, 1], format='csc')
Dso = d * L
Kso = k * L
Bso = np.zeros((n, 1))
Bso[0, 0] = 1
Cso = np.zeros((1, n))
Cso[0, -1] = 1

In [ ]:
E = sps.block_diag([sps.eye(n), Mso], format='csc')
A = sps.bmat([[None, sps.eye(n)], [-Kso, -Dso]], format='csc')
B = np.vstack([np.zeros((n, 1)), Bso])
C = np.hstack([Cso, np.zeros((1, n))])

### `LTIModel`

In [ ]:
from pymor.models.iosys import LTIModel

In [ ]:
LTIModel?

In [ ]:
fom = LTIModel.from_matrices(A, B, C, E=E)

In [ ]:
fom

In [ ]:
print(fom)

## Full-Order Model Analysis

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['axes.grid'] = True

In [ ]:
from pymor.core.logger import set_log_levels
set_log_levels({'pymor.algorithms.gram_schmidt.gram_schmidt': 'ERROR'})

### Impulse Response

The general solution is
$$
x(t) = \int_0^t e^{\tau E^{-1} A} E^{-1} B u(t - \tau) \, \mathrm{d}\tau.
$$
Therefore, the output is
$$
y(t) = \int_0^t C e^{\tau E^{-1} A} E^{-1} B u(t - \tau) \, \mathrm{d}\tau.
$$
I.e.,
$$y = h * u$$
where
$$
h(t) =
\begin{cases}
  C e^{\tau E^{-1} A} E^{-1} B, & t \ge 0, \\
  0, & t < 0.
\end{cases}
$$
The function $h$ is called the *impulse response*.

In [ ]:
from pymor.algorithms.timestepping import ImplicitEulerTimeStepper

In [ ]:
fom2 = fom.with_(T=6000, time_stepper=ImplicitEulerTimeStepper(6000))

In [ ]:
h = fom2.impulse_resp()

In [ ]:
h.shape

In [ ]:
fig, ax = plt.subplots()
_ = ax.plot(np.linspace(0, fom2.T, fom2.time_stepper.nt + 1), h[:, 0, 0])
_ = ax.set(
    xlabel='Time $t$ (s)',
    ylabel='$h(t)$',
    title='Impulse response',
)

### Transfer Function

Applying the Laplace transform to $y = h * u$ gives
$$Y = H U$$
where $U, Y, H$ are respectively the Laplace transforms of $u, y, h$.

In particular,
$$
H(s) = C (s E - A)^{-1} B.
$$
The function $H$ is called the *transfer function*.

### Bode Plot

It can be checked that for $u(t) = \sin(\omega t + \varphi) e_j$
(where $e_j \in \mathbb{R}^m$ is the $j$th canonical vector),
we have that
$$
y_i(t)
\sim
\lvert H_{ij}(\imag \omega) \rvert
\sin\bigl(\omega t + \varphi + \arg(H_{ij}(\imag \omega))\bigr)
$$
as $t \to \infty$.

The Bode plot shows
the magnitudes $\omega \mapsto \lvert H_{ij}(\imag \omega) \rvert$ and
phases $\omega \mapsto \arg(H_{ij}(\imag \omega))$.

In [ ]:
w = (5e-4, 5e-2)
_ = fom.transfer_function.bode_plot(w)

### Hankel Singular Values

The *Hankel operator* $\mathcal{H} \colon L_2((-\infty, 0]) \to L_2([0, \infty))$,
given by
$$
\mathcal{H}(u)(t) = \int_{-\infty}^0 h(t - \tau) u(t) \, \mathrm{d}\tau,
$$
is of finite rank.
It has at most $n$ positive singular values
$$\sigma_1 \ge \sigma_2 \ge \ldots \ge \sigma_n \ge 0.$$
The system is *minimal* iff $\sigma_n > 0$.

It turns out that $\sigma_1^2, \sigma_2^2, \ldots, \sigma_n^2$ are the eigenvalues of $E^{\operatorname{T}} Q E P$,
where
$$
\begin{align*}
  P
  & =
  \int_0^\infty
  e^{\tau E^{-1} A} E^{-1} B
  B\tran E\mtran e^{\tau A\tran E\mtran}
  \, \mathrm{d}\tau, \\
  E^{\operatorname{T}} Q E
  & =
  \int_0^\infty
  e^{\tau A\tran E\mtran} C
  C\tran e^{\tau E^{-1} A}
  \, \mathrm{d}\tau.
\end{align*}
$$
The matrix $P$ is called the *controllability Gramian*.
The matrix $E^{\operatorname{T}} Q E$ is called the *observability Gramian*.

The matrices $P$ and $Q$ are solutions to Lyapunov equations
$$
\begin{align*}
  A P E\tran
  + E P A\tran
  + B B\tran
  & = 0, \\
  A\tran Q E
  + E\tran Q A
  + C\tran C
  & = 0,
\end{align*}
$$

In [ ]:
hsv = fom.hsv()

In [ ]:
fig, ax = plt.subplots()
_ = ax.semilogy(range(1, len(hsv) + 1), hsv, '.-')
_ = ax.set(
    xlabel='Index $i$',
    ylabel=r'$\sigma_i$',
    title='Approximate Hankel singular values'
)

## Balanced Truncation

Balanced truncation constructs matrices $V, W \in \mathbb{R}^{n \times r}$
and the reduced-order model
$$
\begin{align*}
  \hat{E} \dot{\hat{x}}(t) & = \hat{A} \hat{x}(t) + \hat{B} u(t), \quad \hat{x}(0) = 0, \\
  \hat{y}(t) & = \hat{C} \hat{x}(t),
\end{align*}
$$
by Petrov-Galerkin projection
$$
\begin{align*}
  \hat{E} = W\tran E V,\
  \hat{A} = W\tran A V,\
  \hat{B} = W\tran B,\
  \hat{C} = C V,
\end{align*}
$$
Note that
- $u$ is the same input as in the full-order model,
- $\hat{x}(t) \in \mathbb{R}^r$ is the reduced state,
- $\hat{y}(t) \in \mathbb{R}^p$ is the approximate output,
- $\hat{E}, \hat{A} \in \mathbb{R}^{r \times r}$,
- $\hat{B} \in \mathbb{R}^{r \times m}$,
- $\hat{C} \in \mathbb{R}^{p \times r}$.

In [ ]:
from pymor.reductors.bt import BTReductor

In [ ]:
BTReductor?

In [ ]:
bt = BTReductor(fom)

In [ ]:
bt.reduce?

In [ ]:
r = 14
rom_bt = bt.reduce(r)

The reduced-order model is again an `LTIModel`.

In [ ]:
rom_bt

In [ ]:
print(rom_bt)

### Poles

The system poles are the poles of the tranfer function $H(s) = C (s E - A)^{-1} B$.
Assuming there are no zero-pole cancellations (true when the system is minimal),
the poles are the eigenvalues of $E^{-1} A$.

The transfer function of the reduced-order model is
$\hat{H}(s) = \hat{C} (s \hat{E} - \hat{A})^{-1} \hat{B}$.

In [ ]:
poles_rom_bt = rom_bt.poles()

In [ ]:
fig, ax = plt.subplots()
_ = ax.plot(poles_rom_bt.real, poles_rom_bt.imag, 'x')
_ = ax.set(
    xlabel='Real',
    ylabel='Imag',
    title='Poles of the BT ROM',
)

### Impulse Response

In [ ]:
rom_bt2 = rom_bt.with_(T=6000, time_stepper=ImplicitEulerTimeStepper(6000))

In [ ]:
h_bt = rom_bt2.impulse_resp()

In [ ]:
h_bt.shape

In [ ]:
fig, ax = plt.subplots()
_ = ax.plot(np.linspace(0, fom2.T, fom2.time_stepper.nt + 1), h[:, 0, 0], label='FOM')
_ = ax.plot(np.linspace(0, rom_bt2.T, rom_bt2.time_stepper.nt + 1), h_bt[:, 0, 0], '--', label='BT ROM')
_ = ax.set(
    xlabel='Time $t$ (s)',
    ylabel='$h(t)$',
    title='Impulse response',
)
_ = ax.legend()

### Bode Plot

In [ ]:
fig, ax = plt.subplots(2, 1, squeeze=False, figsize=(6, 8), tight_layout=True)
_ = fom.transfer_function.bode_plot(w, ax=ax, label='FOM')
_ = rom_bt.transfer_function.bode_plot(w, ax=ax, linestyle='--', label='BT ROM')
_ = ax[0, 0].legend()
_ = ax[1, 0].legend()

### Hankel Singular Values

Balanced truncation preserves the top $r$ Hankel singular values.

In [ ]:
hsv_bt = rom_bt.hsv()

In [ ]:
fig, ax = plt.subplots()
_ = ax.semilogy(range(1, len(hsv) + 1), hsv, '.-')
_ = ax.semilogy(range(1, len(hsv_bt) + 1), hsv_bt, '.-')
_ = ax.set(
    xlabel='Index $i$',
    ylabel=r'$\sigma_i$',
    title='Approximate Hankel singular values'
)

### Error Magnitude Plot

The error system transfer function
$$H_{\text{err}} = H - \hat{H}$$
has a state-space realization
$$
\begin{align*}
  \begin{bmatrix}
    E & 0 \\
    0 & \hat{E}
  \end{bmatrix}
  \begin{bmatrix}
    \dot{x}(t) \\
    \dot{\hat{x}}(t)
  \end{bmatrix}
  & =
  \begin{bmatrix}
    A & 0 \\
    0 & \hat{A}
  \end{bmatrix}
  \begin{bmatrix}
    x(t) \\
    \hat{x}(t)
  \end{bmatrix}
  +
  \begin{bmatrix}
    B \\
    \hat{B}
  \end{bmatrix}
  u(t), \\
  y(t) - \hat{y}(t)
  & =
  \begin{bmatrix}
    C & -\hat{C}
  \end{bmatrix}
  \begin{bmatrix}
    x(t) \\
    \hat{x}(t)
  \end{bmatrix}.
\end{align*}
$$

In [ ]:
err_bt = fom - rom_bt

The magnitude plot only shows the mapping $\omega \mapsto \lVert H(\imag \omega) \rVert$.

In [ ]:
fig, ax = plt.subplots()
_ = err_bt.transfer_function.mag_plot(w, ax=ax)
_ = ax.set_title('Error magnitude plot')

### Relative $\mathcal{H}_2$ Error

One norm over the space of transfer functions is the $\mathcal{H}_2$ norm:
$$
\lVert H \rVert_{\mathcal{H}_2}
=
\left(
\frac{1}{2 \pi}
\int_{-\infty}^{\infty}
\lVert H(\imag \omega) \rVert_{\operatorname{F}}^2
\,\mathrm{d}\omega
\right)^{1/2}
$$
It provides an time-domain output error bound
$$
\lVert y - \hat{y} \rVert_{L_{\infty}}
\le
\lVert H - \hat{H} \rVert_{\mathcal{H}_2}
\lVert u \rVert_{L_2}.
$$
The relative $\mathcal{H}_2$ error
$$
\frac{\lVert H - \hat{H} \rVert_{\mathcal{H}_2}}{\lVert H \rVert_{\mathcal{H}_2}}
$$
is then a good measure of reduced-order model accuracy.

In [ ]:
print(err_bt.h2_norm() / fom.h2_norm())

### $\mathcal{H}_\infty$ Error Bounds

The $\mathcal{H}_\infty$ norm of a transfer function $H$ is
$$
\lVert H \rVert_{\mathcal{H}_\infty}
= \sup_{\omega \in \mathbb{R}} \lVert H(\imag \omega) \rVert_2.
$$
It also provides an time-domain output error bound
$$
\lVert y - \hat{y} \rVert_{L_2}
\le
\lVert H - \hat{H} \rVert_{\mathcal{H}_{\infty}}
\lVert u \rVert_{L_2}.
$$
Note the difference compared to the $\mathcal{H}_2$ norm.

Generally, for any transfer function $\hat{H}$ of a system of order $r$,
we have the lower bound for the $\mathcal{H}_\infty$ error:
$$
\lVert H - \hat{H} \rVert_{\mathcal{H}_\infty}
\ge \sigma_{r + 1}.
$$
For reduced-order models obtained by balanced truncation,
we also have the upper bound
$$
\lVert H - \hat{H} \rVert_{\mathcal{H}_\infty}
\le 2 \sum_{i = r + 1}^n \sigma_i.
$$

In [ ]:
error_bounds = bt.error_bounds()

In [ ]:
len(error_bounds)

In [ ]:
fig, ax = plt.subplots()
_ = ax.semilogy(range(1, len(error_bounds) + 1), hsv[1:len(error_bounds)+1], '.-')
_ = ax.semilogy(range(1, len(error_bounds) + 1), error_bounds, '.-')
_ = ax.set(
    xlabel='Reduced order',
    title=r'Approximate lower and upper bounds for the $\mathcal{H}_\infty$ error',
)

## Exercise 1: Iterative Rational Krylov Algorithm (IRKA)

IRKA is a method for locally $\mathcal{H}_2$-optimal model order reduction.
It is based on the necessary $\mathcal{H}_2$-optimality conditions
(stated here for SISO systems, i.e., when $m = p = 1$)
$$
\begin{align*}
  H\!\left(-\overline{\lambda_i}\right)
  & = \hat{H}\!\left(-\overline{\lambda_i}\right) \\
  H'\!\left(-\overline{\lambda_i}\right)
  & = \hat{H}'\!\left(-\overline{\lambda_i}\right)
\end{align*}
$$
where $\hat{H}(s) = \sum_{i = 1}^r \frac{\varphi_i}{s - \lambda_i}$
is a locally $\mathcal{H}_2$-optimal reduced-order transfer function of order $r$
with pairwise distinct poles.

The above interpolation can be achieved
using Petrov-Galerkin projection
via rational Krylov subspaces,
but the interpolation points $\lambda_i$ are not known in advance,
which motivates a fixed-point iteration:

1. Choose initial interpolation points $\sigma_i$.
2. Find $\hat{H}$ that interpolates $H$ at $\sigma_i$.
3. Update $\sigma_i$ as the mirrored poles of $\hat{H}$.
4. If $\sigma_i$ changed significantly, go to step 2.

In [ ]:
from pymor.reductors.h2 import IRKAReductor

In [ ]:
IRKAReductor?

Instatiate the IRKA reductor

In [ ]:
irka = ...

In [ ]:
irka.reduce?

Use `irka` to find a reduced-order model with the following as the initial interpolation points.

In [ ]:
sigma = np.logspace(-4, -2, r//2)
sigma = np.hstack((1j * sigma, -1j * sigma))

In [ ]:
rom_irka = irka.reduce(sigma)

In [ ]:
rom_irka

In [ ]:
print(rom_irka)

Analyse the reduced-order model.

### Poles

### Impulse Response

### Bode Plot

### Error Magnitude Plot

### Relative $\mathcal{H}_2$ Error

## `SecondOrderModel`

Note that the reduced-order model from BT (and IRKA)
does not preserve the second-order structure
present in the mass-spring-damper system.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4), constrained_layout=True)
Ebt, Abt = rom_bt.E.matrix, rom_bt.A.matrix
vmin = min(Ebt.min(), Abt.min())
vmax = min(Ebt.max(), Abt.max())
_ = ax[0].imshow(Ebt, vmin=vmin, vmax=vmax)
_ = ax[0].set_title('E')
out = ax[1].imshow(Abt, vmin=vmin, vmax=vmax)
_ = ax[1].set_title('A')
_ = fig.colorbar(out)
_ = fig.suptitle('BT ROM matrices')

To use reductors that preserve the second-order structure, we need to form a `SecondOrderModel`.

In [ ]:
from pymor.models.iosys import SecondOrderModel

In [ ]:
SecondOrderModel?

In [ ]:
som = SecondOrderModel.from_matrices(Mso, Dso, Kso, Bso, Cso)

In [ ]:
som

In [ ]:
print(som)

### Position/Velocity Singular Values

As a heuristic, "Hankel" singular values based on the

- position controllability Gramian $P_p$,
- velocity controllability Gramian $P_v$,
- position observability Gramian $Q_p$, and
- velocity observability Gramian $M_{so}\tran Q_v M_{so}$,

can be used to chose the reduced order,
where
$$
P =
\begin{bmatrix}
  P_p & P_{12} \\
  P_{12}\tran & P_v
\end{bmatrix}
\quad \text{and} \quad
Q =
\begin{bmatrix}
  Q_p & Q_{12} \\
  Q_{12}\tran & Q_v
\end{bmatrix}.
$$

In [ ]:
psv = som.psv()
vsv = som.vsv()
pvsv = som.pvsv()
vpsv = som.vpsv()

In [ ]:
fig, axs = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(8, 6), constrained_layout=True)
ax = axs[0, 0]
_ = ax.semilogy(range(1, len(psv) + 1), psv, '.-')
_ = ax.set_title('Position singular values')
ax = axs[0, 1]
_ = ax.semilogy(range(1, len(vsv) + 1), vsv, '.-')
_ = ax.set_title('Velocity singular values')
ax = axs[1, 0]
_ = ax.semilogy(range(1, len(pvsv) + 1), pvsv, '.-')
_ = ax.set_title('Position-velocity singular values')
ax = axs[1, 1]
_ = ax.semilogy(range(1, len(vpsv) + 1), vpsv, '.-')
_ = ax.set_title('Velocity-position singular values')

## Exercise 2: Second-Order Balanced Truncation

In [ ]:
from pymor.reductors.sobt import SOBTpReductor

In [ ]:
SOBTpReductor?

In [ ]:
sobtp = ...

In [ ]:
sobtp.reduce?

In [ ]:
rom_sobtp = sobtp.reduce(r//2)

The reduced-order model is now a `SecondOrderModel`.

In [ ]:
rom_sobtp

In [ ]:
print(rom_sobtp)

### Poles

### Impulse Response

### Bode Plot

### Error Magnitude Plot

### Relative $\mathcal{H}_2$ Error